## Step 1: Load dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip ../input/facial-keypoints-detection/training.zip
!unzip ../input/facial-keypoints-detection/test.zip

In [ ]:
train = pd.read_csv("./training.csv")
test = pd.read_csv("./test.csv")
print(train.shape) 
print(test.shape)

In [ ]:
train.isnull().sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.scatter(x=train.loc[:, 'left_eye_center_x'],         y=train.loc[:, 'left_eye_center_y'],         marker="<", c='r', label='left_eye',      alpha=0.3)
ax.scatter(x=train.loc[:, 'right_eye_center_x'],        y=train.loc[:, 'right_eye_center_y'],        marker=">", c='r', label='right_eye',     alpha=0.3)
ax.scatter(x=train.loc[:, 'left_eyebrow_outer_end_x'],  y=train.loc[:, 'left_eyebrow_outer_end_y'],  marker="<", c='b', label='left_eyebrow',  alpha=0.3)
ax.scatter(x=train.loc[:, 'right_eyebrow_outer_end_x'], y=train.loc[:, 'right_eyebrow_outer_end_y'], marker=">", c='b', label='right_eyebrow', alpha=0.3)
ax.scatter(x=train.loc[:, 'nose_tip_x'],                y=train.loc[:, 'nose_tip_y'],                marker="o", c='g', label='nose',          alpha=0.3)
ax.scatter(x=train.loc[:, 'mouth_center_bottom_lip_x'], y=train.loc[:, 'mouth_center_bottom_lip_y'], marker="o", c='m', label='mouth',         alpha=0.3)

ax.invert_yaxis()
ax.grid(0)
ax.legend()

plt.show()

In [ ]:
def plot_fig(image, keypoint):
    fig, ax = plt.subplots(1, 1)
    ax.imshow(image, cmap='gray')
    ax.scatter(x=keypoint[np.arange(0, len(keypoint), 2)], y=keypoint[np.arange(1, len(keypoint), 2)], marker='+', c='r')
    ax.axis('off')
    plt.show()

In [ ]:
def to_image(image):
    return np.array(image.split(' '), dtype=int).reshape(96, 96)

In [ ]:
data = train.iloc[0]
keypoint = data.iloc[:-1].to_numpy()
image = to_image(data.Image)

plot_fig(image=image, keypoint=keypoint)

## Step 2: Data cleaning

In [ ]:
x_train = np.array([to_image(train.Image[i]) for i in range(len(train))]) / 255
x_test = np.array([to_image(test.Image[i]) for i in range(len(test))]) / 255

In [ ]:
y_train = train.iloc[:, :-1]

In [ ]:
filter = np.sum(y_train.notnull(), axis=1) == 30

In [ ]:
x_train_clean = x_train[filter]
y_train_clean = y_train[filter]

In [ ]:
x_train_clean, x_val_clean, y_train_clean, y_val_clean = train_test_split(x_train_clean, y_train_clean, 
                                                                          test_size=0.2, random_state=1234)

In [ ]:
x_train.shape, y_train.shape, x_val_clean.shape, y_val_clean.shape

## Step 3: Data augmentation

In [ ]:
from skimage.transform import SimilarityTransform, warp, rotate, resize
from skimage.exposure import adjust_gamma
from skimage.util import random_noise
import math

In [ ]:
plot_fig(x_train_clean[0], y_train_clean.iloc[0])

In [ ]:
def DataAugmentation(x, y, angle, translation, factor, gamma, var):
    y = y.reshape(int(len(y)/2), 2)
    # random_noise
    x = random_noise(x, var=var)
    # adjust_gamma
    x = adjust_gamma(x, gamma)
    # rotation
    x = rotate(x, angle)
    new_coor = []
    for i in y:
        px, py = i
        qx = 48 + math.cos(-math.radians(angle)) * (px - 48) - math.sin(-math.radians(angle)) * (py - 48)
        qy = 48 + math.sin(-math.radians(angle)) * (px - 48) + math.cos(-math.radians(angle)) * (py - 48)
        new_coor.append([qx, qy])
    y = new_coor

    # translation
    tform = SimilarityTransform(translation=translation)
    x = warp(x, tform)
    new_coor = []
    for i in y:
        px, py = i
        qx = px - translation[0]
        qy = py - translation[1]
        new_coor.append([qx, qy])
    y = new_coor
    
    # zoom
    a = np.zeros((96, 96))
    _factor = (1+factor[0], 1+factor[1])
    residue = ( round((-factor[0]*96)/2), round((-factor[1]*96)/2) )
    image = resize(x, output_shape=( round(_factor[1]*96), round(_factor[0]*96) ))
    if factor[0] < 0 and factor[1] < 0:
        a[residue[1]:residue[1]+image.shape[0], residue[0]:residue[0]+image.shape[1]] = image
    elif factor[0] < 0 and factor[1] >= 0:
        x_ = round((image.shape[0]-96)/2)
        a[:, residue[0]:residue[0]+image.shape[1]] = image[x_:x_+96, :]
    elif factor[0] >= 0 and factor[1] < 0:
        y_ = round((image.shape[1]-96)/2)
        a[residue[1]:residue[1]+image.shape[0], :] = image[:, y_:y_+96]
    else:
        x_ = round((image.shape[0]-96)/2)
        y_ = round((image.shape[1]-96)/2)
        a = image[x_:x_+96, y_:y_+96]
    new_coor = []
    for i in y:
        px, py = i
        qx = 48 + (px-48)*_factor[0]
        qy = 48 + (py-48)*_factor[1]
        new_coor.append([qx, qy])
    return a, np.array(new_coor).flatten()

In [ ]:
x_train_clean_aug, y_train_clean_aug = DataAugmentation(x_train_clean[0], 
                                                        y_train_clean.iloc[0].to_numpy(), 
                                                        angle=10, 
                                                        translation=(5, 5), 
                                                        factor=(0.1, 0.1),
                                                        gamma=0.5,
                                                        var=0.003)

plot_fig(x_train_clean_aug, y_train_clean_aug)

In [ ]:
angle_range = range(-30, 31, 1)
translation_range = range(-10, 11, 1)
factor_range = [-0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5]
gamma_range = [0.5, 0.75, 1.0, 1.25, 1.5]
var_range = [0.003, 0.002, 0.001, 0]

x_train_clean_aug = []
y_train_clean_aug = []

for times in range(5):
    for i in range(len(x_train_clean)):
        x, y = x_train_clean[i], y_train_clean.iloc[i].to_numpy()
        x, y = DataAugmentation(x, y, 
                                angle=random.choice(angle_range),
                                translation=(random.choice(translation_range), random.choice(translation_range)),
                                factor=(random.choice(factor_range), random.choice(factor_range)),
                                gamma=random.choice(gamma_range),
                                var=random.choice(var_range))
        if y.min() > 0 and y.max() < 96:
            x_train_clean_aug.append(x)
            y_train_clean_aug.append(y)

In [ ]:
x_train_clean_aug = np.array(x_train_clean_aug)
y_train_clean_aug = np.array(y_train_clean_aug)

In [ ]:
x_train_clean.shape, y_train_clean.shape, x_train_clean_aug.shape, y_train_clean_aug.shape

In [ ]:
x_train_clean_aug = np.concatenate((x_train_clean_aug, x_train_clean), axis=0)
y_train_clean_aug = np.concatenate((y_train_clean_aug, y_train_clean.to_numpy()), axis=0)

In [ ]:
x_train_clean_aug.shape, y_train_clean_aug.shape, x_val_clean.shape, y_val_clean.shape

In [ ]:
# to 3 channel
x_train_clean = np.expand_dims(x_train_clean, axis=-1)
x_train_clean_aug = np.expand_dims(x_train_clean_aug, axis=-1)
x_val_clean = np.expand_dims(x_val_clean, axis=-1)

In [ ]:
x_train_clean = np.tile(x_train_clean, [1, 1, 1, 3])
x_train_clean_aug = np.tile(x_train_clean_aug, [1, 1, 1, 3])
x_val_clean = np.tile(x_val_clean, [1, 1, 1, 3])

## Step 4: Build model

In [ ]:
preprocess_input = tf.keras.applications.resnet_v2.preprocess_input
base_model = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(96, 96, 3),
                                                        include_top=False,
                                                        weights='imagenet',
                                                        classifier_activation=None)

inputs = tf.keras.Input(shape=(96, 96, 3))

# ResNet50V2
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# Dense
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
outputs = tf.keras.layers.Dense(30)(x)

In [ ]:
model = tf.keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss=tf.keras.losses.MeanSquaredError(name='MSE'),
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='RMSE')])

## Step 5: Train model

In [ ]:
my_callbacks = tf.keras.callbacks.EarlyStopping(patience=30, 
                                                monitor='val_loss', 
                                                restore_best_weights=True)

history = model.fit(x_train_clean_aug, 
                    y_train_clean_aug, 
                    epochs=500, 
                    batch_size=16, 
                    validation_data=(x_val_clean, y_val_clean), 
                    callbacks=[my_callbacks])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.set_title('RMSE')
ax1.plot(history.history['RMSE'], 'r', label='RMSE')
ax1.plot(history.history['val_RMSE'], 'g', label='val_RMSE')
ax1.legend()
ax1.grid(axis='x')

ax2.set_title('Loss')
ax2.plot(history.history['loss'], 'r', label='loss')
ax2.plot(history.history['val_loss'], 'g', label='val_loss')
ax2.legend()
ax2.grid(axis='x')

plt.show()

## Step 6: Show high MSE data

In [ ]:
y_pred = model.predict(x_train_clean)
MSE = tf.math.reduce_mean(tf.square(y_train_clean - y_pred), axis=1)
_filter = MSE.numpy() > 5
_filter.sum()

In [ ]:
x, y = 5, (len(x_train_clean[_filter])//5+1)
plt.figure(figsize=(x*3, y*3))
for i in range(len(x_train_clean[_filter])):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_train_clean[_filter][i], cmap='gray')
    keypoint = y_train_clean[_filter].iloc[i]
    plt.scatter(x=keypoint[np.arange(0, len(keypoint), 2)], y=keypoint[np.arange(1, len(keypoint), 2)], marker='+', c='r')
    keypoint_pred = y_pred[_filter][i]
    plt.scatter(x=keypoint_pred[np.arange(0, len(keypoint_pred), 2)], y=keypoint_pred[np.arange(1, len(keypoint_pred), 2)], marker='2', c='b')
    plt.axis('off')
plt.show()

In [ ]:
np.where(_filter==True)

In [ ]:
y_pred = model.predict(x_val_clean)
MSE = tf.math.reduce_mean(tf.square(y_val_clean - y_pred), axis=1)
_filter = MSE.numpy() > 5
_filter.sum()

In [ ]:
x, y = 5, (len(x_val_clean[_filter])//5+1)
plt.figure(figsize=(x*3, y*3))
for i in range(len(x_val_clean[_filter])):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_val_clean[_filter][i], cmap='gray')
    keypoint = y_val_clean[_filter].iloc[i]
    plt.scatter(x=keypoint[np.arange(0, len(keypoint), 2)], y=keypoint[np.arange(1, len(keypoint), 2)], marker='+', c='r')
    keypoint_pred = y_pred[_filter][i]
    plt.scatter(x=keypoint_pred[np.arange(0, len(keypoint_pred), 2)], y=keypoint_pred[np.arange(1, len(keypoint_pred), 2)], marker='2', c='b')
    plt.axis('off')
plt.show()

In [ ]:
np.where(_filter==True)

## Step 7: Submit the results

In [ ]:
x_test = np.expand_dims(x_test, axis=-1)
x_test = np.tile(x_test, [1, 1, 1, 3])

In [ ]:
preds = model.predict(x_test)

In [ ]:
idlookup_file = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
feature_names = list(idlookup_file['FeatureName'])
image_ids = list(idlookup_file['ImageId']-1)
row_ids = list(idlookup_file['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(preds[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)